In [1]:
import matplotlib.pylab as plt

In [2]:
# plt.imread('demo/1.jpg').shape

In [3]:
import os
import numpy as np
import argparse
import torch
# from src.dataset import CocoDataset
from torch.utils.data import DataLoader
# from src.transform import SSDTransformer
import cv2
import shutil

from src.utils import generate_dboxes, Encoder, colors
from src.model import SSD, ResNet

import src
import albumentations as A
import torchmetrics

from torchmetrics.detection.mean_ap import MeanAveragePrecision

/home/sk/repo/SSD_pytorch/src/process.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/home/sk/.local/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/sk/.local/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/sk/.local/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/sk/.local/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnorma

In [ ]:
# _map = MeanAveragePrecision(box_format="xyxy", class_metrics=True)
# _map.update()

In [4]:
import os
import functools, fnmatch, itertools
import pathlib
def find_files(dir_path: str=None, patterns: [str]=None) -> [str]:
    """
    Returns a generator yielding files matching the given patterns
    :type dir_path: str
    :type patterns: [str]
    :rtype : [str]
    :param dir_path: Directory to search for files/directories under. Defaults to current dir.
    :param patterns: Patterns of files to search for. Defaults to ["*"]. Example: ["*.json", "*.xml"]
    """
    path = dir_path or "."
    path_patterns = patterns or ["*"]
    fn = []
    for root_dir, dir_names, file_names in os.walk(path):

        filter_partial = functools.partial(fnmatch.filter, file_names)

        for file_name in itertools.chain(*map(filter_partial, path_patterns)):
             fn.append(pathlib.Path(os.path.join(root_dir, file_name)))

    return sorted(fn)

In [5]:
model = SSD(backbone=ResNet(), num_classes=2)

In [6]:
checkpoint = torch.load('/home/sk/SSD-pytorch/trained_models/Invivo_heavy_aug_train/best_SSD_Invivo_heavy_aug_train.pth')
model.load_state_dict(checkpoint["model_state_dict"])

if torch.cuda.is_available():
    model.cuda()
model.eval()
dboxes = generate_dboxes(model="ssd")
encoder = Encoder(dboxes)

In [7]:
# img_path = '/home/sk/Rewire_Image/Rewire_original_models/c-fos/test/'

# img_path_list = find_files(img_path, ['*.jpg'])
# target_path_list = find_files(img_path, ['*.json'])


In [8]:

Invivo_val_transform = A.Compose([
    A.LongestMaxSize(300),
    A.PadIfNeeded(300,300, border_mode=0)],
    bbox_params=A.BboxParams(format='coco', min_area=15, min_visibility=0.1, label_fields=['class_labels']))


In [10]:
train_path = '/home/sk/repo/FishCam/splitted_anno_0/'

train_set = src.coco_dataset.CocoTestDataset(train_path, 'val', dboxes, img_aug = Invivo_val_transform)
train_loader = DataLoader(train_set, batch_size = 16, shuffle = False, num_workers = 20)

output_path = 'Invivo_heavy_aug_test'

if os.path.isdir(output_path):
    shutil.rmtree(output_path)
os.makedirs(output_path)

nms_threshold = 0.2
cls_threshold = 0.0005

total_19_predictions = []
total_20_predictions = []

for i, (img, img_id, img_size, gtbox, gtlabel) in enumerate(train_set):
    if img is None:
        continue
    if torch.cuda.is_available():
        img = img.cuda()

    
    with torch.no_grad():
        ploc, plabel = model(img.unsqueeze(dim=0))
            
        result = encoder.decode_batch(ploc, plabel, nms_threshold, max_output= 30)[0]
        loc, label, prob = [r.cpu().numpy() for r in result]
        best = np.argwhere(prob > cls_threshold).squeeze(axis=1)
        loc = loc[best]
        label = label[best]
        prob = prob[best]
        file_name = train_loader.dataset.coco.dataset['images'][i]['file_name']
        if len(loc) > 0:
            np_img = img.permute(1, 2, 0).cpu().numpy()
            opencv_img = cv2.cvtColor(cv2.normalize(np_img, None, 255, 0, cv2.NORM_MINMAX, cv2.CV_8U), cv2.COLOR_RGB2BGR)
            # path = test_set.coco.loadImgs(img_id)[0]["file_name"]
            height, width= np_img.shape[:2]
            loc[:, 0::2] *= width
            loc[:, 1::2] *= height
            gtbox = np.array(gtbox, dtype = np.uint32)
            gtbox[:, 2:] = gtbox[:, :2] + gtbox[:, 2:]
            
            loc = loc.astype(np.int32)
            if file_name.find('19') != -1:
                print(file_name)
                total_19_predictions.append(loc)
            else:
                total_20_predictions.append(loc)
            # gtbox = gtbox.int()
            
            for box, lb, pr in zip(loc, label, prob):
                
                category = 'Fish'
                
                color = colors[lb]
                xmin, ymin, xmax, ymax = box
                opencv_img= cv2.rectangle(opencv_img, (xmin, ymin), (xmax, ymax), color, 2)


            for box in gtbox:
                
                category = 'GT'
                
                color = colors[4]
                xmin, ymin, xmax, ymax = box
                opencv_img= cv2.rectangle(opencv_img, (xmin, ymin), (xmax, ymax), color, 1)
                # text_size = cv2.getTextSize(category + " : %.2f" % pr, cv2.FONT_HERSHEY_PLAIN, 0.01, 1)[0]
                # cv2.rectangle(opencv_img, (xmin, ymin), (xmin + text_size[0] + 3, ymin + text_size[1] + 4), color, 1)
                # cv2.putText(
                #     opencv_img, category + " : %.2f" % pr,
                #     (xmin, ymin + text_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1,
                #     (255, 255, 255), 1)
            cv2.imwrite("{}/{}_prediction.jpg".format(output_path, i), opencv_img)


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Invivo/sampled_images/sampled_low_images/111023_19fish_tank6_video2/111023_19fish_tank6_video2_00007.png
Invivo/sampled_images/sampled_low_images/111023_19fish_tank6_video2/111023_19fish_tank6_video2_00004.png
Invivo/sampled_images/sampled_low_images/111023_19fish_tank6_video2/111023_19fish_tank6_video2_00003.png
Invivo/sampled_images/sampled_low_images/111023_19fish_tank6_video2/111023_19fish_tank6_video2_00001.png
Invivo/sampled_images/sampled_low_images/111023_19fish_tank6_video1/111023_19fish_tank6_video1_00006.png
Invivo/sampled_images/sampled_low_images/111023_19fish_tank5_video2/111023_19fish_tank5_video2_00007.png
Invivo/sampled_images/sampled_low_images/111023_19fish_tank5_video1/111023_19fish_tank5_video1_00000.png
Invivo/sampled_images/sampled_low_images/111023_19fish_tank4_video2/111023_19fish_tank4_video2_00006.png
Invivo/sampled_images/sampled_low_images/111023_19fish_tank4_video2/111023_19

In [ ]:
train_loader.dataset.coco.dataset['images'][0]['file_name'].find('19')

In [ ]:
pred_19 = np.array([len(f) for f in total_19_predictions])

In [ ]:
pred_20 = np.array([len(f) for f in total_20_predictions])

In [ ]:
import pandas as pd

In [ ]:
test_results = pd.DataFrame([pred_19, pred_20]).T

In [ ]:
test_results.columns = ['19', '20']

In [ ]:
test_results.describe()

In [ ]:
len(total_20_predictions)

In [ ]:
train_loader.dataset.coco.dataset['images'][0]

In [ ]:
_map = MeanAveragePrecision(box_format="xyxy", class_metrics=True)
# _map.update()

In [ ]:
loc

In [ ]:
_map.update(preds=loc, target=gtbox)

In [ ]:
train_loader.dataset.coco.dataset['annotations'][0]

In [ ]:
[f for f in train_loader]

In [ ]:
for i in train_set:
    print(i)
    break

In [ ]:
box

In [ ]:
loc[0]

In [ ]:
gtbox.numpy()

In [ ]:
gtbox.shape

In [ ]:
gtlabel.shape

In [ ]:
plabel.shape

In [ ]:
s_result = encoder.decode_batch(gtbox, gtlabel, nms_threshold, max_output= 30)[0]

In [ ]:
n = 0
for box in gtbox.numpy():
    
    xmin, ymin, xmax, ymax = box
    try:
        opencv_img =cv2.rectangle(opencv_img, (xmin, ymin), (xmax, ymax), colors[3], 1)
    except:
        print('except: ', n)
        n+=1
        pass


In [ ]:
train_loader.dataset.coco.dataset

In [ ]:
plt.imshow(opencv_img)

In [ ]:
loc

In [ ]:
torch.zeros([2, 4], dtype=torch.int32)

In [ ]:
targets = []
    for i in range(batch_size):
        targets.append(
            dict(
                boxes=annotations["bbox"][i][:, [1, 0, 3, 2]],
                labels=annotations["cls"][i],
            )
        )
# self.map.update(preds=preds, target=targets)

In [ ]:
height

In [ ]:
loc

In [ ]:
gtbox[:, 0::2]

In [ ]:
gtbox[:, 0::2] *=width
gtbox[:, 1::2] *=height

In [ ]:
gtbox

In [ ]:
gtlabel

In [ ]:
loc

In [ ]:
img_id

In [ ]:
import numpy as np
from tqdm.autonotebook import tqdm
import torch
from pycocotools.cocoeval import COCOeval
# from apex import amp


In [ ]:
train_path = '/home/sk/repo/FishCam/splitted_anno_0/'

train_set = src.coco_dataset.CocoTestDataset(train_path, 'val', dboxes, img_aug = Invivo_val_transform)
train_loader= DataLoader(train_set, batch_size = 10, shuffle = False, num_workers = 0)

In [ ]:
train_loader.dataset.label_info

In [ ]:
train_loader.dataset.coco.loadRes?

In [ ]:
COCOeval?

In [ ]:
category_ids = train_loader.dataset.coco.getCatIds()


In [ ]:
category_ids

In [ ]:
img_size[1]

In [ ]:
def evaluate(model, test_loader, epoch, writer, encoder, nms_threshold):
    model.eval()
    detections = []
    category_ids = test_loader.dataset.coco.getCatIds()
    for nbatch, (img, img_id, img_size, _, _) in enumerate(test_loader):
        print("Parsing batch: {}/{}".format(nbatch, len(test_loader)), end="\r")
        if torch.cuda.is_available():
            img = img.cuda()
        with torch.no_grad():
            # Get predictions
            ploc, plabel = model(img)
            ploc, plabel = ploc.float(), plabel.float()

            for idx in range(ploc.shape[0]):
                ploc_i = ploc[idx, :, :].unsqueeze(0)
                plabel_i = plabel[idx, :, :].unsqueeze(0)
                try:
                    result = encoder.decode_batch(ploc_i, plabel_i, nms_threshold, 200)[0]
                except:
                    print("No object detected in idx: {}".format(idx))
                    continue

                height, width = img_size[idx]
                loc, label, prob = [r.cpu().numpy() for r in result]
                for loc_, label_, prob_ in zip(loc, label, prob):
                    detections.append([img_id[idx], loc_[0] * width, loc_[1] * height, (loc_[2] - loc_[0]) * width,
                                       (loc_[3] - loc_[1]) * height, prob_,
                                       category_ids[label_ - 1]])

    detections = np.array(detections, dtype=np.float32)

    coco_eval = COCOeval(test_loader.dataset.coco, test_loader.dataset.coco.loadRes(detections), iouType="bbox")
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    writer.add_scalar("Test/mAP", coco_eval.stats[0], epoch)


In [ ]:
tr